# Toronto Neighborhoods Project

First, we will import all libraries and modules needed.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

We are going to use pandas read_html() method to read a wikipedia Table and save it into a dataframe called pd.

In [2]:
#We will also look at the head of the dataframe
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


# Data pre-processing

As we have to ignore the rows without an assigned borough, we will delete them using the folowing cell:

In [3]:
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


We also have to group the neighborhood with the same postal code, but by watching the whole table there are not cases like that, there are some rows with multiple Neighborhoods separated by a "/".

We will convert each column to a list, so we can work easier with them.

In [4]:
pc=df['Postal code'].values.tolist()
b=df['Borough'].values.tolist()
n=df['Neighborhood'].values.tolist()

We are going to replace al the sub-strings " / " with ", " in the Neighborhood list, as the homework says.

In [5]:
for i in range (0, 103):
    n[i] = n[i].replace(' / ', ', ')

Now, we will use the three lists to create a new dataframe called toronto, we will keep using the same column names as the ones in the original table. 

In [6]:
toronto=pd.DataFrame({'Postal code':pc, 'Borough':b, 'Neighborhood':n})
toronto.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


As the geocoder module and the code in the homework description didn't work for me, i searched and found a dataset with the latitude and longitude of all the postal codes we need, we will use it. As this dataset called Geospatial_Coordinates.csv is ordered by the postal code values, we will have to apply the same to our toronto dataset

In [7]:
toronto=toronto.sort_values(by=['Postal code'])

In [8]:
#This cell is for reading the csv file into a dataframe called df_data_1, I will hide mi personal account information with X, but whole cell
#was generated by IBM Watson Studio to import a csv file
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We will have to reset the index of our toronto dataframe in order to avoid mistakes while adding the new 2 columns

In [9]:
toronto.reset_index(drop=True, inplace=True)

We will now simply copy the Latitude and Longitude colums from the csv to new colums on the toronto dataframe using the same names for the columns

In [10]:
toronto['Latitude']=df_data_1['Latitude']
toronto['Longitude']=df_data_1['Longitude']

In [11]:
toronto.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Exploring the data

To explore the data, we will use the same process as the one in the lab example, the only change is that we are going to use the Borough of Scarborough

In [12]:
import json
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from pandas.io.json import json_normalize

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

In [13]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
map_ = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_)  
    
map_

In [15]:
scar_data = toronto[toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scar_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
address = 'Scarborough, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [17]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
LIMIT=50
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e92a6579fcb92001c0a40a5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c059bcd7083952134097bce',
       'name': 'SEPHORA',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'at Scarborough Town Centre',
        'lat': 43.77501688366838,
        'lng': -79.25810909472256,
       

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,SEPHORA,Cosmetics Shop,43.775017,-79.258109
1,Disney Store,Toy / Game Store,43.775537,-79.256833
2,American Eagle Outfitters,Clothing Store,43.776012,-79.258334
3,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
4,Tommy Hilfiger,Clothing Store,43.776015,-79.257369


In [23]:
scar_venues = getNearbyVenues(names=scar_data['Neighborhood'],
                                   latitudes=scar_data['Latitude'],
                                   longitudes=scar_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [60]:
scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",6,6,6,6,6,6
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",7,7,7,7,7,7
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [24]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Café,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
scar_onehot.shape

(95, 58)

In [26]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Café,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.20,0.000000,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.25,...,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000
2,Cedarbrae,0.000000,0.111111,0.000000,0.111111,0.111111,0.00,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000000,0.000000,0.000000,0.071429,0.00,0.000000,0.000000,0.00,...,0.142857,0.00,0.000000,0.000000,0.071429,0.00,0.000000,0.000000,0.071429,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000000,0.000000,0.222222,0.000000,0.00,0.000000,0.222222,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.142857,0.000000,0.00,...,0.000000,0.00,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,"Malvern, Rouge",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [27]:
num_top_venues = 5

for hood in scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1               Skating Rink   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0      Thai Restaurant  0.11
1               Bakery  0.11
2                 Bank  0.11
3  Fried Chicken Joint  0.11
4               Lounge  0.11


----Clarks Corners, Tam O'Shanter, Sullivan----
                venue  freq
0            Pharmacy  0.14
1         Pizza Place  0.14
2   Convenience Store  0.07
3  Chinese Restaurant  0.07
4         Gas Station  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1                Mot

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Golf Course,General Entertainment
1,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,Golf Course,Gas Station,Fried Chicken Joint
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Convenience Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Pharmacy,Noodle House,Chinese Restaurant,Thai Restaurant,Gas Station,Shopping Mall,Bank,Fried Chicken Joint,Italian Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Movie Theater,Motel,Grocery Store,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [30]:
# set number of clusters
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 4, 4, 4, 4, 4, 1, 2], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scar_merged = scar_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scar_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Construction & Landscaping,Vietnamese Restaurant,Home Service,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4.0,History Museum,Golf Course,Bar,Home Service,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Intersection,Bank,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Construction & Landscaping,Golf Course
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Indian Restaurant,Korean Restaurant,College Stadium,Grocery Store,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Convenience Store


In [32]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
scar_merged['Cluster Labels'] = scar_merged['Cluster Labels'].fillna(0)
scar_merged['Cluster Labels'] = scar_merged['Cluster Labels'].astype(int)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters